In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - Environment Setup

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/00_environment_setup.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/00_environment_setup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/00_environment_setup.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

Before you run this notebook, make sure that you have completed the steps in [README](README.md).

In this notebook, you will setup your environment for Fraudfinder to be used in subsequent labs.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)
- [Google Cloud Storage](https://cloud.google.com/storage)
- [Pub/Sub](https://cloud.google.com/pubsub/)

Steps performed in this notebook:

- Setup your environment.
- Load historical bank transactions into BigQuery.
- Read data from BigQuery tables.
- Read data from Pub/Sub topics, which contain a live stream of new transactions.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Pub/Sub
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Pub/Sub pricing](https://cloud.google.com/pubsub/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Install additional packages

Install the following packages required to execute this notebook.

In [2]:
# ! pip install --upgrade -r 'requirements.txt'

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup your environment

Run the next cells to import libraries used in this notebook and configure some options.

Run the next cell to set your project ID and some of the other constants used in the lab.  

In [6]:
import random
import string
from typing import Union

import pandas as pd
from google.cloud import bigquery

# Generate unique ID to help w/ unique naming of certain pieces
ID = "".join(random.choices(string.ascii_lowercase + string.digits, k=5))

GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
REGION = "us-central1"
TRAINING_DS_SIZE = 1000

In [7]:
PROJECT_ID

'my-practice-setting'

### Create a Google Cloud Storage bucket and save the config data.

Next, we will create a Google Cloud Storage bucket and will save the config data in this bucket. After the cell operation finishes, you can navigate to [Google Cloud Storage](https://console.cloud.google.com/storage/) to see the GCS bucket. 

In [3]:
config = f"""
BUCKET_NAME          = \"{BUCKET_NAME}\"
PROJECT              = \"{PROJECT_ID}\"
REGION               = \"{REGION}\"
ID                   = \"{ID}\"
FEATURESTORE_ID      = \"fraudfinder_{ID}\"
MODEL_NAME           = \"ff_model\"
ENDPOINT_NAME        = \"ff_model_endpoint\"
TRAINING_DS_SIZE     = \"{TRAINING_DS_SIZE}\"
"""

!gsutil mb -l {REGION} gs://{BUCKET_NAME}

!echo '{config}' | gsutil cp - gs://{BUCKET_NAME}/config/notebook_env.py

Creating gs://my-practice-setting-fraudfinder/...
Copying from <STDIN>...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              


### Copy the historical transaction data into BigQuery tables

Now we will copy the historical transaction data and ingest it into BigQuery tables. For this, we will need to run `copy_bigquery_data.py`.

In [4]:
!python3 scripts/copy_bigquery_data.py $BUCKET_NAME

File copied from gs://cymbal-fraudfinder/datagen/hacked_customers_history.txt 
		 to gs://my-practice-setting-fraudfinder/datagen/hacked_customers_history.txt
File copied from gs://cymbal-fraudfinder/datagen/hacked_terminals_history.txt 
		 to gs://my-practice-setting-fraudfinder/datagen/hacked_terminals_history.txt
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_profiles.csv 
		 to gs://my-practice-setting-fraudfinder/datagen/demographics/customer_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/terminal_profiles.csv 
		 to gs://my-practice-setting-fraudfinder/datagen/demographics/terminal_profiles.csv
File copied from gs://cymbal-fraudfinder/datagen/demographics/customer_with_terminal_profiles.csv 
		 to gs://my-practice-setting-fraudfinder/datagen/demographics/customer_with_terminal_profiles.csv
BigQuery table created: `my-practice-setting`.tx.tx
BigQuery table created: `my-practice-setting`.tx.txlabels
BigQuery table created: `my-pra

### Check data in BigQuery

After ingesting our data into BigQuery, it's time to run some queries against the tables to inspect the data. You can also go to the [BigQuery console](https://console.cloud.google.com/bigquery) to see the data.

#### Initialize BigQuery SDK for Python 

Use a helper function for sending queries to BigQuery.

In [2]:
import sys
sys.path.append('./scripts')  # Add the 'scripts' folder to Python path

from scripts import copy_bigquery_data as cbd

cbd.run_bq_query(
"""
SELECT
  *
FROM
  tx.tx
LIMIT 5
"""
)

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,c576a00f41db1ed760015abe0a37f8468a09841e,2022-05-18 16:43:12+00:00,4885882308700125,00064542,53.600000000
1,6cbddf54087adec4bd6ac1c8ab7a0c7bae6ebdc7,2022-05-18 11:19:51+00:00,1671597784093246,00064542,102.110000000
2,e68badd7b5337a73d561c99bb1487c9102ea092f,2022-05-18 15:03:19+00:00,4247660428380717,00064542,32.570000000
3,3a54514dce9bad062aa3093dc5f68f100f8fe2f5,2022-05-18 09:48:22+00:00,4188530197004811,00064542,45.600000000
4,8633547653650deda11877ab5f28db507672e022,2022-05-18 10:44:08+00:00,9277495820614995,00064542,52.420000000


#### tx.tx
The `tx.tx` table contains the basic information about each transaction:
- `TX_ID` is a unique ID per transaction
- `TX_TS` is the timestamp of the transaction, in UTC
- `CUSTOMER_ID` is a unique 16-digit string ID per customer
- `TERMINAL_ID` is a unique 16-digit string ID per point-of-sale terminal
- `TX_AMOUNT` is the amount of money spent by the customer at a terminal, in dollars

#### tx.txlabels
The `tx.txlabels` table contains information on whether each transation was fraud or not:
- `TX_ID` is a unique ID per transaction
- `TX_FRAUD` is 1 if the transaction was fraud, and 0 if the transaction was not fraudulent

In [3]:
cbd.run_bq_query(
    """
SELECT
  *
FROM
  tx.txlabels
LIMIT 5
"""
)

,TX_ID,TX_FRAUD
0,0b1e0244a325b29ff71ea429903968cf636f0a06,0
1,4d5db022844983c9d98f516469aa0b76e5479088,0
2,810d0eec5028f72819ad992be54214d6f1e91ad5,0
3,42afff6de1c6b638a6d4f640928bae2784daf961,0
4,46a6601c03e40fcc5e978a8d3ebb82b9118d550a,0


### Check live streaming transactions via public Pub/Sub topics

As part of the [README](README.md), you've created [subscriptions](https://console.cloud.google.com/cloudpubsub/subscription/) to public Pub/Sub topics, where there is a constant flow of new transactions. This means you have, in your own Google Cloud project, subscriptions to the public Pub/Sub topics. You will receive a Pub/Sub message in your subscription every time a new transaction is streamed into the Pub/Sub topic.

There are two public Pub/Sub topics where there is a constant stream of live transactions occurring.

The following Pub/Sub topics are used for transactions:
```
projects/cymbal-fraudfinder/topics/ff-tx
projects/cymbal-fraudfinder/topics/ff-txlabels
```

Note: If you haven't completed the steps in the README, please make sure that you complete them first before continuing this notebook, otherwise you may not have Pub/Sub subscriptions.

### Reading messages from the Pub/Sub topics

#### Reading from the `ff-tx-sub` subscription

Now let's read from the `ff-tx-sub` subscription. You should see some recent transactions (in UTC timezone).

In [17]:
# Added here to refresh and use the new function - read_from_sub
%load_ext autoreload
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
messages_tx = cbd.read_from_sub(
    project_id=PROJECT_ID, subscription_name="ff-tx-sub", messages=2
)

messages_tx

Received and acknowledged 2 messages from projects/my-practice-setting/subscriptions/ff-tx-sub.


[{'TX_ID': '9ca1c751cd758b4a31cf73dbae8c433894c03ce9',
  'TX_TS': '2025-07-19 18:26:55',
  'CUSTOMER_ID': '9337390360570867',
  'TERMINAL_ID': '06778120',
  'TX_AMOUNT': 76.6},
 {'TX_ID': '41ae5f0a4087c72cdf62ff6bc0a22efdfca4fafb',
  'TX_TS': '2025-07-19 18:28:24',
  'CUSTOMER_ID': '5195027930521056',
  'TERMINAL_ID': '85199517',
  'TX_AMOUNT': 46.66}]

#### Reading from the `ff-txlabels-sub` subscription

We will do the same with `ff-txlabels-sub` subscription, which receives the same stream of transactions as `ff-tx-sub`, but also contain the ground-truth label, `TX_FRAUD`, if the transaction is fraudulent (1) or not (0).

In [10]:
messages_txlabels = cbd.read_from_sub(
    project_id=PROJECT_ID, subscription_name="ff-txlabels-sub", messages=2
)

messages_txlabels

Received and acknowledged 2 messages from projects/my-practice-setting/subscriptions/ff-txlabels-sub.


[{'TX_ID': '5b1fdcb9bebe468b04ec835df46b5def9cca3691', 'TX_FRAUD': 0},
 {'TX_ID': '53557d1d0c3189689faf50422e37d2020ef89caf', 'TX_FRAUD': 0}]